In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from games import rock_paper_scissors, chicken_or_dare

In [ ]:
from agents import MWagent

In [ ]:
def simulate_game(T, agents, game):
    num_agents = len(agents)
    all_weights = np.ones((num_agents, T, agents[0].num_actions))
    all_scores = np.ones((num_agents, T))
    assert(len(all_scores) == num_agents)
    for t in range(T):
        agent_actions = {}
        for i, agent in enumerate(agents):
            agent_actions[f'p{i + 1}_action'] = agent.get_action()
        
        scores = game(**agent_actions)
        
        assert(len(scores) == num_agents)
        
        for i, agent in enumerate(agents):
            agent.update_weights(agent_actions[f'p{i + 1}_action'], scores[i])
            all_weights[i, t, :] = agent.weights.copy()
            all_scores[i, t] = scores[i]
    return all_weights, all_scores

# Rock, Paper, Scissors experiments

In [ ]:
gamma = 0.01
p1 = MWagent(
    n_actions=3,
    gamma=gamma,
)
p2 = MWagent(
    n_actions=3,
    gamma=gamma,
)

T = 10000

weights, scores = simulate_game(T, [p1, p2], rock_paper_scissors)

In [ ]:
plt.figure()
plt.plot(weights[0, :, 0], label='p1 0')
plt.plot(weights[0, :, 1], label='p1 1')
plt.plot(weights[0, :, 2], label='p1 2')
plt.legend()
plt.plot()

In [ ]:
plt.figure()
plt.plot(weights[1, :, 0], label='p2 0')
plt.plot(weights[1, :, 1], label='p2 1')
plt.plot(weights[1, :, 2], label='p2 2')
plt.legend()
plt.plot()

# Chicken or Dare

In [ ]:
gamma = 0.001
p1 = MWagent(
    n_actions=2,
    gamma=gamma,
)
p2 = MWagent(
    n_actions=2,
    gamma=gamma,
)

T = 10000

weights, scores = simulate_game(T, [p1, p2], chicken_or_dare)

In [ ]:
plt.figure()
plt.plot(weights[0, :, 0], label='p1 0')
plt.plot(weights[0, :, 1], label='p1 1')
plt.legend()
plt.plot()

In [ ]:
plt.figure()
plt.plot(weights[1, :, 0], label='p2 0')
plt.plot(weights[1, :, 1], label='p2 1')
plt.legend()
plt.plot()